# 簡単に作れる　Graph Transformer 

**Transformer** [(Vaswani et al. 2017)](https://proceedings.neurips.cc/paper/2017/hash/3f5ee243547dee91fbd053c1c4a845aa-Abstract.html) は、自然言語処理とコンピュータビジョンにおいて効果的な学習アーキテクチャであることが証明されている。近年、Transformerをグラフ学習応用するような試みが盛んに行われていおり、多くの実践的なタスク、例えばグラフ特性予測において成功を収めつつある。 [Dwivedi et al. (2020)](https://arxiv.org/abs/2012.09699) は、最初にTransformerのニューラルアーキテクチャをグラフ構造データに一般化した。ここでは、DGLのsparse matrix APIを用いてそのようなGraph Transformerを構築する方法を紹介する。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dmlc/dgl/blob/master/notebooks/sparse/graph_transformer.ipynb) [![GitHub](https://img.shields.io/badge/-View%20on%20GitHub-181717?logo=github&logoColor=ffffff)](https://github.com/dmlc/dgl/blob/master/notebooks/sparse/graph_transformer.ipynb)

In [1]:
# パッケージのインストール
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

# Uncomment below to install required packages. If the CUDA version is not 11.8,
# check the https://www.dgl.ai/pages/start.html to find the supported CUDA
# version and corresponding command to install DGL.
#!pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html > /dev/null
#!pip install ogb >/dev/null

try:
    import dgl
    installed = True
except ImportError:
    installed = False
print("DGL installed!" if installed else "Failed to install DGL!")

DGL installed!


## Sparse Multi-head Attention

Transformerにおける全ペアのスケールドット積アテンションメカニズムを思い出してほしい:

$$\text{Attn}=\text{softmax}(\dfrac{QK^T} {\sqrt{d}})V,$$

一方でグラフトランスフォーマー（GT）モデルは、Sparse Multi-head Attention ブロックを採用している:

$$\text{SparseAttn}(Q, K, V, A) = \text{softmax}(\frac{(QK^T) \circ A}{\sqrt{d}})V,$$

ここで、 $Q, K, V ∈\mathbb{R}^{N\times d}$ はそれぞれクエリ特徴、キー特徴、およびバリュー特徴である。 $A\in[0,1]^{N\times N}$ は入力グラフの隣接行列である。 $(QK^T)\circ A$ は、クエリ行列とキー行列の積の後に疎隣接行列とのアダマール積（要素ごとの積）が行われることを意味する。下図に示されているように:

<p align="center">
<img src="./images/sparseattn.png" width="500">
</p>

本質的には、$A$ の疎性に従って接続されたノード間のアテンションスコアのみが計算される。この操作は *Sampled Dense Dense Matrix Multiplication (SDDMM)* とも呼ばれる。  

DGLの [batched SDDMM API](https://docs.dgl.ai/en/latest/generated/dgl.sparse.bsddmm.html) を利用することで、複数のアテンションヘッド（異なる表現部分空間）の計算を並列化することができる。

In [2]:
import dgl
import dgl.nn as dglnn
import dgl.sparse as dglsp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from dgl.data import AsGraphPredDataset
from dgl.dataloading import GraphDataLoader
from ogb.graphproppred import collate_dgl, DglGraphPropPredDataset, Evaluator
from ogb.graphproppred.mol_encoder import AtomEncoder
from tqdm import tqdm

# Sparse Multi-head Attention Moduleの実装
class SparseMHA(nn.Module):
    """Sparse Multi-head Attention Module"""

    def __init__(self, hidden_size=80, num_heads=8):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.scaling = self.head_dim**-0.5

        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, hidden_size)

    def forward(self, A, h):
        N = len(h)
        # [N, dh, nh]
        q = self.q_proj(h).reshape(N, self.head_dim, self.num_heads)
        q *= self.scaling
        # [N, dh, nh]
        k = self.k_proj(h).reshape(N, self.head_dim, self.num_heads)
        # [N, dh, nh]
        v = self.v_proj(h).reshape(N, self.head_dim, self.num_heads)

        ######################################################################
        # (HIGHLIGHT) Compute the multi-head attention with Sparse Matrix API
        ######################################################################
        # dglspのbsddmm関数を使って、SDDMMをバッチ単位で計算する
        attn = dglsp.bsddmm(A, q, k.transpose(1, 0))  # (sparse) [N, N, nh]
        # Sparse softmax by default applies on the last sparse dimension.
        attn = attn.softmax()  # (sparse) [N, N, nh]
        out = dglsp.bspmm(attn, v)  # [N, dh, nh]

        return self.out_proj(out.reshape(N, -1))

## Graph Transformer Layer

GT層は、マルチヘッドアテンション、バッチノーマライゼーション、フィードフォワードネットワークで構成されており、これらは通常のトランスフォーマーのように残差リンクで接続されている。

<p align="center">
<img src="./images/gt_layers.png" width="300">
</p>

In [3]:
class GTLayer(nn.Module):
    """Graph Transformer Layer"""

    def __init__(self, hidden_size=80, num_heads=8):
        super().__init__()
        self.MHA = SparseMHA(hidden_size=hidden_size, num_heads=num_heads)
        self.batchnorm1 = nn.BatchNorm1d(hidden_size)
        self.batchnorm2 = nn.BatchNorm1d(hidden_size)
        self.FFN1 = nn.Linear(hidden_size, hidden_size * 2)
        self.FFN2 = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, A, h):
        h1 = h
        h = self.MHA(A, h)
        h = self.batchnorm1(h + h1)

        h2 = h
        h = self.FFN2(F.relu(self.FFN1(h)))
        h = h2 + h

        return self.batchnorm2(h)

## Graph Transformer Model

GTモデルは、GT層を積み重ねて構築されます。通常のTransformerの入力位置エンコーディングは、ラプラシアン位置エンコーディング[(Dwivedi et al. 2020)](https://arxiv.org/abs/2003.00982)に置き換えられている。グラフレベルの予測タスクのために、同じグラフのノード特徴を集約するためにGT層の上に追加のpooling層が積み重ねられている。

In [4]:
class GTModel(nn.Module):
    def __init__(
        self,
        out_size,
        hidden_size=80,
        pos_enc_size=2,
        num_layers=8,
        num_heads=8,
    ):
        super().__init__()
        self.atom_encoder = AtomEncoder(hidden_size)
        self.pos_linear = nn.Linear(pos_enc_size, hidden_size)
        self.layers = nn.ModuleList(
            [GTLayer(hidden_size, num_heads) for _ in range(num_layers)]
        )
        self.pooler = dglnn.SumPooling()
        self.predictor = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            nn.ReLU(),
            nn.Linear(hidden_size // 4, out_size),
        )

    def forward(self, g, X, pos_enc):
        indices = torch.stack(g.edges())
        N = g.num_nodes()
        A = dglsp.spmatrix(indices, shape=(N, N)) # 疎行列（隣接行列）の作成
        h = self.atom_encoder(X) + self.pos_linear(pos_enc) # 位置エンコーディングを加算
        for layer in self.layers:
            h = layer(A, h) # GTLayerをnum_layers回繰り返す
        h = self.pooler(g, h) # プーリング

        return self.predictor(h)

## 学習

今回は、GTモデルを[ogbg-molhiv](https://ogb.stanford.edu/docs/graphprop/#ogbg-mol) ベンチマークで学習する。各グラフのラプラシアン位置エンコーディングは、モデルへの入力の一部として事前に計算されている（APIは[こちら](https://docs.dgl.ai/en/latest/generated/dgl.laplacian_pe.html)）。

*デモをより高速に実行するために、データセットをダウンサンプリングしていることに注意してください。フルデータセットでのパフォーマンスについては、*[*サンプルスクリプト*](https://github.com/dmlc/dgl/blob/master/examples/sparse/graph_transformer.py)*を参照してください。*

In [5]:
@torch.no_grad()
def evaluate(model, dataloader, evaluator, device):
    model.eval()
    y_true = []
    y_pred = []
    for batched_g, labels in dataloader:
        batched_g, labels = batched_g.to(device), labels.to(device)
        y_hat = model(batched_g, batched_g.ndata["feat"], batched_g.ndata["PE"])
        y_true.append(labels.view(y_hat.shape).detach().cpu())
        y_pred.append(y_hat.detach().cpu())
    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    return evaluator.eval(input_dict)["rocauc"]


def train(model, dataset, evaluator, device):
    train_dataloader = GraphDataLoader(
        dataset[dataset.train_idx],
        batch_size=256,
        shuffle=True,
        collate_fn=collate_dgl,
    )
    valid_dataloader = GraphDataLoader(
        dataset[dataset.val_idx], batch_size=256, collate_fn=collate_dgl
    )
    test_dataloader = GraphDataLoader(
        dataset[dataset.test_idx], batch_size=256, collate_fn=collate_dgl
    )
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 20
    scheduler = optim.lr_scheduler.StepLR(
        optimizer, step_size=num_epochs, gamma=0.5
    )
    loss_fcn = nn.BCEWithLogitsLoss()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batched_g, labels in train_dataloader:
            batched_g, labels = batched_g.to(device), labels.to(device)
            logits = model(
                batched_g, batched_g.ndata["feat"], batched_g.ndata["PE"]
            )
            loss = loss_fcn(logits, labels.float())
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
        avg_loss = total_loss / len(train_dataloader)
        val_metric = evaluate(model, valid_dataloader, evaluator, device)
        test_metric = evaluate(model, test_dataloader, evaluator, device)
        print(
            f"Epoch: {epoch:03d}, Loss: {avg_loss:.4f}, "
            f"Val: {val_metric:.4f}, Test: {test_metric:.4f}"
        )


# Training device.
#dev = torch.device("cpu")
# Uncomment the code below to train on GPU. Be sure to install DGL with CUDA support.
dev = torch.device("cuda:0") # GPUで学習

# データセットの読み込み
pos_enc_size = 8
dataset = AsGraphPredDataset(
    DglGraphPropPredDataset("ogbg-molhiv", "./data/OGB")
)
evaluator = Evaluator("ogbg-molhiv")

# データセットのサンプリング
import random
random.seed(42)
train_size = len(dataset.train_idx)
val_size = len(dataset.val_idx)
test_size = len(dataset.test_idx)
dataset.train_idx = dataset.train_idx[
    torch.LongTensor(random.sample(range(train_size), 2000))
]
dataset.val_idx = dataset.val_idx[
    torch.LongTensor(random.sample(range(val_size), 1000))
]
dataset.test_idx = dataset.test_idx[
    torch.LongTensor(random.sample(range(test_size), 1000))
]

# ラプラシアン位置エンコーディングの計算
indices = torch.cat([dataset.train_idx, dataset.val_idx, dataset.test_idx])
for idx in tqdm(indices, desc="Computing Laplacian PE"):
    g, _ = dataset[idx]
    g.ndata["PE"] = dgl.laplacian_pe(g, k=pos_enc_size, padding=True) # ラプラシアン位置エンコーディング

# モデルの初期化
out_size = dataset.num_tasks
model = GTModel(out_size=out_size, pos_enc_size=pos_enc_size).to(dev)

# 学習の開始
train(model, dataset, evaluator, dev)

Computing Laplacian PE:   0%|          | 0/4000 [00:00<?, ?it/s]/home/takutoyo/.pyenv/versions/miniconda3-latest/envs/dgl_env/lib/python3.11/site-packages/dgl/transforms/functional.py:3725: DGLWarning: dgl.laplacian_pe will be deprecated. Use dgl.lap_pe please.
  dgl_warning("dgl.laplacian_pe will be deprecated. Use dgl.lap_pe please.")
Computing Laplacian PE: 100%|██████████| 4000/4000 [00:08<00:00, 475.61it/s]


Epoch: 000, Loss: 0.4006, Val: 0.3367, Test: 0.3609
Epoch: 001, Loss: 0.2118, Val: 0.3531, Test: 0.3778
Epoch: 002, Loss: 0.1763, Val: 0.4515, Test: 0.4052
Epoch: 003, Loss: 0.1601, Val: 0.5234, Test: 0.4207
Epoch: 004, Loss: 0.1520, Val: 0.6077, Test: 0.4546
Epoch: 005, Loss: 0.1353, Val: 0.6955, Test: 0.5026
Epoch: 006, Loss: 0.1203, Val: 0.6491, Test: 0.4770
Epoch: 007, Loss: 0.1114, Val: 0.7394, Test: 0.5912
Epoch: 008, Loss: 0.1001, Val: 0.6127, Test: 0.3912
Epoch: 009, Loss: 0.0894, Val: 0.7504, Test: 0.6017
Epoch: 010, Loss: 0.0835, Val: 0.7043, Test: 0.5280
Epoch: 011, Loss: 0.0771, Val: 0.7186, Test: 0.4636
Epoch: 012, Loss: 0.0645, Val: 0.6442, Test: 0.3509
Epoch: 013, Loss: 0.0506, Val: 0.7130, Test: 0.3913
Epoch: 014, Loss: 0.0421, Val: 0.7165, Test: 0.4508
Epoch: 015, Loss: 0.0704, Val: 0.5453, Test: 0.4081
Epoch: 016, Loss: 0.0483, Val: 0.6208, Test: 0.4531
Epoch: 017, Loss: 0.0428, Val: 0.6330, Test: 0.3983
Epoch: 018, Loss: 0.0289, Val: 0.7081, Test: 0.4750
Epoch: 019, 